# NLP 6: Sentiment analysis

1.   Importe [**cet ensemble de données**](https://github.com/murpi/wilddata/raw/master/quests/tweets.zip) de tweets dans un DataFrame.
2.   Conserve uniquement les tweets positifs et négatifs (donc tu exclus les `neutrals`). Quel est le pourcentage de tweets positifs/négatifs ?
3.   Crée une fonction que tu appelleras `clean` qui prend pour paramètre une phrase (donc un texte `str`) et qui retourne un texte (`str`) de tokens après application d'un stemmer ou d'un lemmatizer, séparés par des espaces.
Tu peux tester ta fonction avec cette phrase, elle doit retourner quelque chose ressemblant à ce résultat :
```
clean("You are better when I am well.")
>>> "you be well when I be well ."
```
4.   Récupère la liste des stopwords anglais depuis NLTK, et copie-la dans une liste `stopwordsenglish`. Complète ta fonction `clean` pour qu'elle supprime la ponctuation et les stopwords.
5.   Applique cette fonction `clean` à la colonne text de ton DataFrame. Stocke le résultat dans une nouvelle colonne `clean` du DataFrame. (Le traitement peut durer 2 ou 3 minutes)
Tu dois maintenant avoir un DataFrame qui ressemble à celui-ci (moins les pronoms, la ponctuation et peut-être d'autres mots en fonction des stopwords que tu as nettoyé) :
![Screenshot](https://storage.googleapis.com/quest_editor_uploads/0OnLjWvjuJG3AHYyQdoss3KBYzznt5hS.png)
6.   Copie la colonne `clean` dans une Serie `X`, et la colonne `sentiment` dans une Serie `y`. Applique un train-test split avec la taille du jeu d'entrainement à `0.75` avec le `random_state = 32`.
7.   Applique un `CountVectorizer` et entraine des modèles de classification.
8.   Applique un `TfidfVectorizer` et entraine des modèles de classification.
9.   Compare les scores, quels paramètres permettent d'avoir les meilleurs scores ?

Bonus : maintenant c'est à toi d'améliorer ton modèle :

En cherchant des paramètres de modèles : par gridsearch et crossvalidation par exemple ;
En changeant la préparation du texte : par exemple certaines ponctuations peuvent aider le modèle, le point d'exclamation notamment.

## 1. Import

In [45]:
import pandas as pd
import re
import nltk
import spacy 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [2]:
df_raw = pd.read_csv('https://github.com/murpi/wilddata/raw/master/quests/tweets.zip')
display(df_raw.head())

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


## 2. Only positive & negative

In [68]:
df = df_raw[df_raw['sentiment'].isin(['negative', 'positive'])]
display(df.head())
sentiment_percentage = round(df['sentiment'].value_counts(normalize=True) * 100,2)
print(sentiment_percentage)
print(len(df))

,textID,text,selected_text,sentiment
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative
6,6e0c6d75b1,2am feedings for the baby are fun when he is a...,fun,positive


sentiment
positive    52.45
negative    47.55
Name: proportion, dtype: float64
16363


## 3. Fonction tokenization

In [67]:
def clean(text):
    nlp = spacy.load('en_core_web_sm')
    sent_tokens = nlp(text)
    lemma_list = [word.lemma_ for word in sent_tokens]
    txt_clean = " ".join(lemma_list)
    return txt_clean

print(clean("You are better when I am well."))
print(type(clean("You are better when I am well.")))

you be well when I be well .
<class 'str'>


## 4. Stopwords

In [65]:
stopwordsenglish = nltk.corpus.stopwords.words('english')

def clean(text):
    txt_no_punct = re.sub(r'[^\w\s]','',text).lower()
    print(txt_no_punct)
    word_tokenized = nltk.word_tokenize(txt_no_punct)
    stopwords_removed = [word for word in word_tokenized if word not in stopwordsenglish]
    nlp = spacy.load('en_core_web_sm')
    sent_tokens = nlp(" ".join(stopwords_removed))
    lemma_list = [word.lemma_ for word in sent_tokens]
    txt_clean = " ".join(lemma_list)
    return txt_clean

print(clean("You are better when I am well."))
print(type(clean("You are better when I am well.")))

sons of  why couldnt they put them on t
son could not put
you are better when i am well
<class 'str'>


## 5. Colonne clean

In [69]:
df_clean = df.copy()
df_clean['clean'] = df_clean['text'].apply(lambda x: clean(x) if pd.notnull(x) else x)



df_clean.head(10)

KeyboardInterrupt: 

## 6. Train-Test Split

In [ ]:
X = df_clean['clean']
y = df_clean['sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    train_size=0.75,
                                                    random_state=32)

## 7. CountVectorizer

In [ ]:
vectorizer = CountVectorizer()
X_train_CV = vectorizer.fit_transform(X_train)

X_test_CV = vectorizer.transform(X_test)
X_test_CV

log_model = LogisticRegression(max_iter=1000).fit(X_train_CV, y_train)

## 8. TfidfVectorizer

In [ ]:
tfidf = TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(X_train)

X_test_tfidf = tfidf.transform(X_test)
X_test_tfidf

log_model = LogisticRegression(max_iter=1000).fit(X_train_tfidf, y_train)

## 9. Comparaisons Scores

In [ ]:
print('CV | accuracy score on train set :',round(log_model.score(X_train_CV, y_train),3))
print('CV | accuracy score on test set :',round(log_model.score(X_test_CV, y_test),3))

print('TFIDF | accuracy score on train set :',round(log_model.score(X_train_tfidf, y_train),3))
print('TFIDF | accuracy score on test set :',round(log_model.score(X_test_tfidf, y_test),3))

## Bonus : amélioration